In [1]:
import Dataset as ds
import config as cfg

from sklearn.model_selection import train_test_split
import os
from tqdm.auto import tqdm

In [3]:
# Base paths for the images and labels
train_images_path = f'{cfg.BASE_DATASET}/images/train'
train_labels_path = f'{cfg.BASE_DATASET}/labels/train'
val_images_path = f'{cfg.BASE_DATASET}/images/valid'
val_labels_path = f'{cfg.BASE_DATASET}/labels/valid'

# Base path for metadata
metadata_path = f'{cfg.BASE_DATASET}/metadata.json'

# Create the DataFrames for the train and validation sets
train_df = ds.create_dataframe(train_images_path, train_labels_path, metadata_path)
valid_df = ds.create_dataframe(val_images_path, val_labels_path, metadata_path)

In [4]:
dataset_name = 'test100'
dataset_dir = cfg.CLF_DATASET_DIR + f'/{dataset_name}'
class_names = cfg.CLF_CLASS_NAMES


_, test_train_df = train_test_split(
    train_df,
    test_size=100,  # Number of items you want in your sample
    stratify=train_df['ac'],  # Stratify based on the combined column
    #random_state=42  # Ensures reproducibility
)

_, test_val_df = train_test_split(
    valid_df,
    test_size=20,  # Number of items you want in your sample
    stratify=valid_df['ac'],  # Stratify based on the combined column
    #random_state=42  # Ensures reproducibility
)

ds.create_sub_dataset(dataset_dir, test_train_df, test_val_df, class_names)


Removing dataset if pre-existing
Copying training files:


Copying files:   0%|          | 0/200 [00:00<?, ?it/s]

Copying validation files:


Copying files:   0%|          | 0/40 [00:00<?, ?it/s]

Dataset 'test100' created at C:\github\Third-Year-Project\Intruder-Aircraft-Detection\datasets\Custom\test100


In [5]:
ds.correct_dataset_labels(dataset_dir, test_train_df, test_val_df, class_names)

Processing train labels in test100:


Processing labels:   0%|          | 0/100 [00:00<?, ?it/s]

Processing valid labels in test100:


Processing labels:   0%|          | 0/20 [00:00<?, ?it/s]

Label correction completed.


In [6]:

augmentation_metadata = {
    'methods': {        
        'flip': {
            'parameters': {
                'orientation': 'h',  # Could be 'h' for horizontal or 'v' for vertical
                'p': 1.0  # Probability of applying the augmentation
            },
            'apply_to_percentage': 0.5  # 50% of the training images
        }        
    }
}

ds.augment_dataset(dataset_dir, augmentation_metadata)

Removing existing directories/files:   0%|          | 0/3 [00:00<?, ?it/s]

Copying files:   0%|          | 0/100 [00:00<?, ?it/s]

Copying files:   0%|          | 0/100 [00:00<?, ?it/s]

Applying augmentations:   0%|          | 0/50 [00:00<?, ?it/s]